## Copyright 2021 Antoine Simoulin.

<i>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

Icons made by <a href="https://www.flaticon.com/authors/freepik" title="Freepik">Freepik</a>, <a href="https://www.flaticon.com/authors/pixel-perfect" title="Pixel perfect">Pixel perfect</a>, <a href="https://www.flaticon.com/authors/becris" title="Becris">Becris</a>, <a href="https://www.flaticon.com/authors/smashicons" title="Smashicons">Smashicons</a>, <a href="https://www.flaticon.com/authors/srip" title="srip">srip</a>, <a href="https://www.flaticon.com/authors/adib-sulthon" title="Adib">Adib</a>, <a href="https://www.flaticon.com/authors/flat-icons" title="Flat Icons">Flat Icons</a> and <a href="https://www.flaticon.com/authors/dinosoftlabs" title="Pixel perfect">DinosoftLabs</a> from <a href="https://www.flaticon.com/" title="Flaticon"> www.flaticon.com</a></i>

# TP 3 : Words Embeddings

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP3%20-%20Word%20Embeddings/tp3-header.png?raw=True" width="1000">

On va s'appuyer sur le corpus collecté par <span class="badge badge-secondary">([Panckhurst et al., 2016](#panckhurst-2016))</span> qui rassemble 88,000 sms collectés dans la région de Montpellier. Le corpus a été dé-identifié (en particulier, les noms sont remplacés par [ _forename_ ]). Pour chaque sms, on a identifié les Emojis dans le texte.

Il y avait beaucoup de type d'Emojis. Dans le TP, ils ont été simplifiés selon le tableau suivant. Tous les Emojis de la colonne `Emoji list` ont été remplacé par l'emoji de la colonne `Generic`. Dans le TP les Emojis n'apparaissent pas dans le texte du sms car on cherche à les prédire.


| Generic Emoji  | Emoji list                                                         |
|:--------------:|:------------------------------------------------------------------:|
| 😃             | '=P', ':)', ':P', '=)', ':p', ':d', ':-)', '=D', ':D', '^^'        |
| 😲             | ':O', 'o_o', ':o', ':&'                                            | 
| 😔             | '"-.-'''", '<_>', '-_-', "--'", "-.-'", '-.-', "-.-''", "-\_-'"    | 
| 😠             | ':/', ':-/', ':-(', ':(', ':-<'                                    | 
| 😆             | '>.<', '¤.¤', '<>','><', '*.*', 'xd', 'XD', 'xD', 'x)',';)', ';-)' | 
| 😍             | '</3', '<3'                                                        | 
 

Finalement pour le TP, on a filtré le jeu de données pour ne conserver que les sms contenant qu'un seul Emoji. On a par ailleurs <i>down samplé</i> les classes majoritaires pour limiter le déséquilibre du jeu de données. En effet les sms avec un smiley 😃 était largement sur-représentés.

<b>L'objet du TP est de prédire l'émoji associé à chaque message. Pour cela on vectorisera le texte en utilisant les méthodes d'embeddings.</b>

In [ ]:
%%capture

# Check environment
if 'google.colab' in str(get_ipython()):
  IN_COLAB = True
else:
  IN_COLAB = False

if IN_COLAB:
  # ⚠️ Execute only if running in Colab
  !pip install -q scikit-learn==0.23.2 matplotlib==3.1.3 pandas==1.1.3 gensim==3.8.1 torch==1.6.0 torchvision==0.7.0
  !pip install skorch==0.10.0
  # then restart runtime environment

In [ ]:
from gensim.models import KeyedVectors

from collections import Counter
import numpy as np
import pandas as pd
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score

import os, sys

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

# Inline Figures with matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# import extrenal modules
import urllib.request

class_names = ['happy', 'joke', 'astonished', 'angry', 'bored', 'heart']
repo_url = 'https://raw.githubusercontent.com/AntoineSimoulin/m2-data-sciences/master/'

_ = urllib.request.urlretrieve(repo_url + 'src/plots.py', 'plots.py')

if not os.path.exists('smileys'):
    os.makedirs('smileys')

for c in class_names:
    _ = urllib.request.urlretrieve(
        repo_url + 'TP3%20-%20Word%20Embeddings/smileys/{}.png'.format(c), 
        'smileys/{}.png'.format(c))

On va utiliser les embeddings déjà entrainé que nous avons manipulé au cours précédent. Pour limiter la taille du fichier d'embeddings, on a sauvegardé que les `10,000` mots les plus fréquents. <b>Vous devez récupérer le fichier d'embeddings aisni que le jeu de données directement sur le [Moodle](https://moodle.u-paris.fr/course/view.php?id=11048).</b>

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format("oscar.fr.300.10k.model")
w2v_model.init_sims(replace=True)

In [ ]:
len(w2v_model.vocab)

In [ ]:
# On crée un array avec les 10,000 premiers mots et on crée le dictionaire de vocabulaire

word_count = {k: w2v_model.vocab[k].count for k in w2v_model.vocab}
word_count = Counter(word_count)
word_count.most_common(10)

idx2w = {i: w for (i, (w, f)) in enumerate(word_count.most_common(10000), 2)}
idx2w[0] = 'unk'
idx2w[1] = 'pad'
w2idx = {w: i for (i, (w, f)) in enumerate(word_count.most_common(10000), 2)}
w2idx['unk'] = 0
w2idx['pad'] = 1

embeddings_vectors = [w2v_model[w] for (w, f) in word_count.most_common(10000)]
word2vec_embeddings = np.vstack(embeddings_vectors)
word2vec_embeddings = np.concatenate((word2vec_embeddings, np.zeros_like(word2vec_embeddings[0:2])), 0)

In [ ]:
word2vec_embeddings.shape

In [ ]:
w2idx['Oh']

In [ ]:
word2vec_embeddings[3664][:10]

In [ ]:
w2v_model['Oh'][:10]

In [ ]:
dataset = pd.read_csv('emojis.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.loc[3, 'sms']

In [ ]:
class_names = ['happy', 'joke', 'astonished', 'angry', 'bored', 'heart']

In [ ]:
dataset.shape

On va utiliser la même fonction de tokenization qui a été utilisée pour entrainer les embeddings.

In [ ]:
token_pattern = re.compile(r"(\->|(?::\)|:-\)|:\(|:-\(|;\);-\)|:-O|8-|:P|:D|:\||:S|:\$|:@|8o\||\+o\(|\(H\)|\(C\)|\(\?\))|(?:[\d.,]+)|([^\s\w0-9])\2*|(?:[\w0-9\.]+['’]?)(?<!\.))")

def tokenize(text):
    tokens = [groups[0] for groups in re.findall(token_pattern, str(text))]
    tokens = [t.strip() for t in tokens]
    return tokens

In [ ]:
dataset['tokens'] = dataset['sms'].apply(tokenize)

In [ ]:
dataset.head()

### Exploration de données

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Observer la distribution des classes.</p>
</div>
<hr>

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Evaluer la proportion de tokens qui sont hors du vocabulaire des embeddings.</p>
</div>
<hr>

### Vectorization

Les embeddings de mots permettent de représenter chaque <i>token</i> par un vecteur. Pour obtenir un vecteur qui représente le sms, on va agréger les différents mots du texte. On considérera plusieurs fonctions d'agrégation : la somme, la moyenne, me maximum ou le minimum.

En pratique nous verrons dans le dernier cours d'ouverture qu'il existe des méthodes plus évoluées pour composer les mots de la phrase. Néanmoins une simple fonction d'agrégation nous donnera déjà une bonne <i>baseline</i>.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP3%20-%20Word%20Embeddings/model.png?raw=True" width="500">

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Ecrire une fonction qui permet de vectoriser un sms.</p>
</div>
<hr>  

In [ ]:
# %load solutions/vectorize_1.py

def vectorize(tokens, agg_method='mean'):
    
    #TODO à compléter
    
    # associer chaque token à son embedding. 
    # Attention, certains tokens peuvent ne pas être dans le vocabulaire
    
    # Agréger les représentations de chaque token.
    # Le vecteur de sortie doit être de taille (300, )
    
    if agg_method == 'mean':
        sentence_embedding = 
    elif agg_method == 'max':
        sentence_embedding = 
    elif agg_method == 'sum':
        sentence_embedding = 
    
    return

In [ ]:
vectorize(dataset['tokens'][0], agg_method='max')

On voudrait attribuer un poids moins important aux embeddings des mots moins caractéristiques. Pour ça, on voudrait pondérer la contribution des vecteurs de chaque mot en fonction de leur score TF-IDF.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP3%20-%20Word%20Embeddings/model-tfidf.png?raw=True" width="700">

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Utiliser la pondération TF-IDF pour pondérer chacun des vecteurs.</p>
</div>
<hr>  

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x,
                                   lowercase=False)

tfidf_vectorizer.fit(dataset['tokens'])

w2idx_tfidf = {w: idx for (idx, w) in enumerate(tfidf_vectorizer.get_feature_names())}
idx_tfidf2w = {idx: w for (idx, w) in enumerate(tfidf_vectorizer.get_feature_names())}

In [ ]:
# %load solutions/vectorize_2.py

def vectorize(tokens, agg_method='mean', tfidf_vectorizer=None):
    
    #TODO à compléter
    
    if agg_method == 'mean':
        sentence_embedding = 
    elif agg_method == 'max':
        sentence_embedding = 
    elif agg_method == 'sum':
        sentence_embedding = 
    elif agg_method == 'tfidf':
        
    
    return

In [ ]:
X = [vectorize(sms) for sms in dataset['tokens']]
X = np.array(X)
print(X.shape)

On va intégrer la fonction `vectorize` dans un module compatible avec les fonctions de `sklearn`.

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Intégrer votre fonction de vectorization dans la classe Vectorizer ci-dessous. Vous devez simoplement la copier/coller en replaçant tfidf_vectorizer par self.tfidf_vectorizer car c'est maintenant un attribut de la class</p>
</div>
<hr>

In [ ]:
# 6 choses à faire pour l'excercice sur la class Vectorizer :
# copier votre fonction vectorize dans la class
# ajouter l'argument self dans la fonction vectorize
# supprimer l'argument tfidf_vectorizer de la fonction vectorize
# remplacer toutes les occurences de agg_method par self.agg_method dans la fonction vectorize
# supprimer l'argument agg_method de la fonction vectorize
# remplacer toutes les occurences de w2idx_tfidf par self.w2idx_tfidf dans la fonction vectorize

In [ ]:
# %load solutions/vectorizer.py

class Vectorizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, agg_method='mean', normalize=False):
        self.agg_method = agg_method
        self.normalize = normalize
        self.tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x,
                                                lowercase=False,
                                                token_pattern=None)
        
    def vectorize(self, tokens):
    
        token_embeddings = []
        vect_vide = np.zeros_like(embeddings_vectors[0])

        # associer chaque token à son embedding. 
        # Attention, certains tokens peuvent ne pas être dans le vocabulaire
        
        # self.tfidf_vectorizer

        for t in tokens:
            if t in w2idx:
                t_idx = w2idx[t]
                token_embeddings.append(embeddings_vectors[t_idx])
            else:
                token_embeddings.append(vect_vide)

        token_embeddings_arr = np.array(token_embeddings)

        # Agréger les représentations de chaque token.
        # Le vecteur de sortie doit être de taille (300, )

        if self.agg_method == 'mean':
            sentence_embedding = np.mean(token_embeddings_arr, axis=0)
        elif self.agg_method == 'max':
            sentence_embedding = np.max(token_embeddings_arr, axis=0)
        elif self.agg_method == 'sum':
            sentence_embedding = np.sum(token_embeddings_arr, axis=0)
        elif self.agg_method == 'tfidf':
            pass

        return sentence_embedding

    
    def _vectorize(self, tokens):
        return vectorize(tokens)
    
    def fit(self, X, y=None): 
        self.tfidf_vectorizer.fit(X['tokens'])
        self.w2idx_tfidf = {w: idx for (idx, w) in enumerate(self.tfidf_vectorizer.get_feature_names())}
        self.idx_tfidf2w = {idx: w for (idx, w) in enumerate(self.tfidf_vectorizer.get_feature_names())}
        return self
    
    def transform(self, X, y=None, eps=1e-12):
        X = [self.vectorize(sms) for sms in X['tokens']]
        X = np.array(X)

        if self.normalize:
            X = X / np.linalg.norm(X + eps, axis=1, keepdims=True)
        return X

In [ ]:
vectorizer = Vectorizer(agg_method='tfidf')

In [ ]:
X = vectorizer.fit_transform(dataset)

In [ ]:
X.shape

### Classification

On compare deux algorithmes de classification :  Une régression logistique et un SVM ou l'on pénalise les classes majoritaires.

In [ ]:
X_train, X_test = train_test_split(
    dataset, test_size=0.33, random_state=42)

y_train = X_train[['happy', 'joke', 'astonished', 'angry', 'bored', 'heart']].astype(int).values
y_train = [x.tolist().index(1) for x in y_train]

y_test = X_test[['happy', 'joke', 'astonished', 'angry', 'bored', 'heart']].astype(int).values
y_test = [x.tolist().index(1) for x in y_test]

In [ ]:
len(y_train)

In [ ]:
X_train.shape

In [ ]:
LogReg_pipeline = Pipeline([
    ('vect', Vectorizer('tfidf')),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'))),
])


# Training logistic regression model on train data
LogReg_pipeline.fit(X_train, y_train)

# Infering data on test set
prediction_LogReg = LogReg_pipeline.predict(X_test)

In [ ]:
SVC_pipeline = Pipeline([
    ('vect', Vectorizer('tfidf')),
    ('clf', OneVsRestClassifier(SVC(kernel='linear', 
                                    class_weight='balanced', # penalize
                                    probability=True), n_jobs=-1))
])


SVC_pipeline.fit(X_train, y_train)
prediction_SVC = SVC_pipeline.predict(X_test)

### Evaluation

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from plots import plot_confusion_matrix

In [ ]:
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction_SVC)))
print('Test ROC socre is {}'.format(roc_auc_score(np.eye(np.max(y_test) + 1)[y_test],
                                                  SVC_pipeline.predict_proba(X_test), 
                                                  multi_class='ovo')))

plot_confusion_matrix(confusion_matrix(y_test, prediction_SVC), 
                      classes=class_names, 
                      title='Confusion matrix, without normalization')

In [ ]:
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction_LogReg)))
print('Test ROC socre is {}'.format(roc_auc_score(np.eye(np.max(y_test) + 1)[y_test],
                                                  LogReg_pipeline.predict_proba(X_test), 
                                                  multi_class='ovo')))

plot_confusion_matrix(confusion_matrix(y_test, prediction_LogReg), 
                      classes=class_names, 
                      title='Confusion matrix, without normalization')

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Quelle mesure de performance vous semble le plus adaptée pour ce cas d'usage ?</p>
</div>
<hr>

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Comparer les résultats obtenus avec les deux algorithmes de classifications</p>
</div>
<hr>

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Comparer les différentes méthodes d'agrégation proposées. (Mean, Max, Sum, Moyenne pondérée par le TF-IDF)</p>
</div>
<hr>

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice (Bonus) :</b> Comparer les résultats obtenus avec un réseau de neurones récurent (RNN).</p>
</div>
<hr>

Nous ferons une ouverture sur les réseaux de neurones et leur utilisation pour le texte lors de la dernière séance. Néanmoins, cela peut être une bonne occasion de se familiariser avec leur utilisation. Nous allons utiliser la librairie [skorch](https://github.com/skorch-dev/skorch) qui est un wrapper de la librairie [PyTorch](https://pytorch.org/) compatible avec [scikit-learn](https://scikit-learn.org/). Cela permet en particulier de simplifier les aspects d'optimisation. Ici on utilise un réseau récurent de type LSTM <span class="badge badge-secondary">([Cho and al., 2014](#cho-2014)</span>, <span class="badge badge-secondary">[Hochreiter and Schmidhuber, 1997](#schmidhuber-1997))</span>. Les réseaux de neurones récurrents modélisent les phrases comme des séquences d’embeddings de mots. Ils traitent l’entrée séquentiellement. A chaque étape, le vecteur de sortie est calculé en fonction de l’embedding du mot courant et de l’état caché précédent.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP3%20-%20Word%20Embeddings/lstm.png?raw=True" width="700">

In [ ]:
import torch
from torch import nn
from skorch import NeuralNet, NeuralNetClassifier
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight


class RNNClassifier(nn.Module):
    def __init__(self, n_classes, embeddings_weights, 
        hidden_dim=100, embedding_dim=300, dropout=0.5):
        super(RNNClassifier, self).__init__()

        self.embeddings = nn.Embedding.from_pretrained(embeddings_weights, sparse=True)
        self.embeddings.weight.requires_grad = False

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(hidden_dim, n_classes)
        self.dropout = dropout
        self.drop = nn.Dropout(self.dropout)

    def forward(self, X, **kwargs):
        X, X_len = X
        X = self.embeddings(X)

        # On utilise une méthode de pytorch pour tenir compte de la longueur des phrases
        # et ainsi s'adapter au padding.
        X_packed = pack_padded_sequence(X, X_len, batch_first=True, enforce_sorted=False)
        X_packed, (h, c) = self.lstm(X_packed)# [1][0] # .transpose(0, 1)
        # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM
        X, output_lengths = pad_packed_sequence(X_packed, batch_first=True)

        # X = torch.sigmoid(X)
        out = F.softmax(self.dense(h.squeeze()), dim=-1)
        return out

class_weights = compute_class_weight(
    'balanced', classes=range(len(class_names)), y=y_train)
# On va donner un poids plus important aux classes minoritaires 
# mais pas proportionnel à leur distribution pour ne pas trop les favoriser 
# au détriment de la précision globale
class_weights = [1, 1, 1.3, 1, 1.3, 1]
class_weights = torch.tensor(class_weights, dtype=torch.float)

net = NeuralNetClassifier( # NeuralNet
    RNNClassifier(len(class_names), torch.tensor(word2vec_embeddings)),
    max_epochs=10,
    lr=0.001,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.NLLLoss,
    criterion__weight=class_weights
) 

sequences = [torch.tensor([w2idx.get(t, 0) for t in tokens]) for tokens in X_train['tokens']]
sequences_length = torch.tensor([len(s) for s in sequences])
# On "pad" les séquences pour qu'elles aient toutes la même longueur.
padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=1)

net.fit((padded_sequences, sequences_length), torch.tensor(y_train))

In [ ]:
sequences_test = [torch.tensor([w2idx.get(t, 0) for t in tokens]) for tokens in X_test['tokens']]
sequences_test_length = torch.tensor([len(s) for s in sequences_test])
# On "pad" les séquences pour qu'elles aient toutes la même longueur.
padded_sequences_test = pad_sequence(sequences_test, batch_first=True, padding_value=1)
prediction_LSTM = net.predict((padded_sequences_test, sequences_test_length))

In [ ]:
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction_LSTM)))
print('Test ROC socre is {}'.format(roc_auc_score(np.eye(np.max(y_test) + 1)[y_test],
                                                  net.predict_proba((padded_sequences_test, sequences_test_length)), 
                                                  multi_class='ovo')))

plot_confusion_matrix(confusion_matrix(y_test, prediction_LSTM), 
                      classes=class_names, 
                      title='Confusion matrix, without normalization')

L'utilisation des méthodes de down-sampling ou up-sampling peut s'avérer fastidieux (on va se priver de données ou en utiliser d'autres plusieurs fois. La sélection des données doit se faire précisémment pour ne pas impacter les capacités de généralisation de l'algorithme). Nous avons préféré ici utiliser un algorithme qui pénalise les classes majoritaires et une mesure d'erreur adaptée. Il existe un bon article de blog pour gérer les classes déséquilibrées : https://elitedatascience.com/imbalanced-classes. 

On peut se faire une idée des limites et des points fort de l'algorithme en regardant des prédictions.

In [ ]:
humors = ['happy', 'astonished', 'bored', 'angry', 'joke', 'heart']
meta_smiley = [b'\xF0\x9F\x98\x83'.decode("utf-8"),
                b'\xF0\x9F\x98\xB2'.decode("utf-8"),
                b'\xF0\x9F\x98\x94'.decode("utf-8"), 
                b'\xF0\x9F\x98\xA0'.decode("utf-8"),
                b'\xF0\x9F\x98\x86'.decode("utf-8"),
                b'\xF0\x9F\x98\x8D'.decode("utf-8")]
humor_2_emoji = {h: ms for (h, ms) in zip(humors, meta_smiley)}

In [ ]:
X_test.shape

In [ ]:
for _ in range(10):
    idx = np.random.randint(0, len(X_test))
    
    emojis = humor_2_emoji[class_names[prediction_SVC[idx]]]
    
    true_emojis = humor_2_emoji[class_names[y_test[idx]]]
    print(X_test['sms'].values[idx], '(Pred)', emojis, '(True)', true_emojis, '\n')
    

### Visualisation

On peut aussi essayer de visualiser plus globalement les représentations. Pour ça on peut utiliser des algorithmes de réduction de dimension pour visualiser nos données. On a déjà parlé de UMAP et t-SNE. De manière intutive, l'algorithme projete les représentations dans un espace de plus faible dimension en s'efforcant de respecter les distances entre les points entre l'espace de départ et d'arrivée. Il permet de visualiser facilement les données. On va utiliser l'outil `Tensorboard` qui intègre les principales méthodes de réduction de dimensions.

In [ ]:
from pathlib import Path
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
from torchvision import transforms

from torch.utils.tensorboard import SummaryWriter
import torch

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
pil_img = Image.open('./smileys/happy.png').convert('RGB')
pil_img = pil_img.resize((100, 100)) 

In [ ]:
smileys_images = [f for f in listdir('./smileys') if isfile(join('./smileys', f))]

In [ ]:
imgs_tb = {}
for s in smileys_images:
    pil_img = Image.open(os.path.join('smileys', s)).convert('RGB')
    pil_img = pil_img.resize((25, 25)) 
    pil_to_tensor = transforms.ToTensor()(pil_img).unsqueeze_(0)
    imgs_tb[Path(os.path.join('smileys', s)).stem] = pil_to_tensor

In [ ]:
writer_embeddings = SummaryWriter(log_dir=os.path.join("./tfb/"))

vectorizer = Vectorizer(agg_method='tfidf', normalize=True)
emb_test = vectorizer.fit_transform(X_test)

writer_embeddings.add_embedding(torch.tensor(emb_test),
                                metadata=[(r, s, l) for (r, s, l) in zip(
                                    X_test['sms'].values,
                                    [humor_2_emoji[class_names[y]] for y in y_test],
                                    [humor_2_emoji[class_names[y]] for y in prediction_SVC])
                                ],
                                label_img=torch.cat([imgs_tb[class_names[y]] for y in y_test]),
                                metadata_header=['sms','label', 'prediction'],
                                tag="SMS-EMB-CLS")

Pour visualiser les représentations, lancer un tensorboard. Dans un terminal, se placer dans le dossier ou est éxécuté le notebook et exécuter:

```
tensorboard --logdir ./tfb/
```

Dans **Colab** on va lancer le tensorboard directement dans le notebook en éxécutant les cellules suivante :

```
%load_ext tensorboard
```

```
%tensorboard --logdir ./tfb/
```

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000);

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Utiliser les méthodes UMAP, PCA et t-SNE pour projeter les données. Comparez les différentes méthodes de projections et interprétez qualitativement les propriétés de vos représentations.</p>
</div>
<hr>

La compatibilité entre Jupyter/Colab et Tensorboard est un parfois instable (c.f. https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks). Si vous êtes sur Colab, vous pouvez télécharger le dossier directement sur votre ordinateur. Téléchargez le .zip, sur votre ordinateur, dezipé le.

```
!zip -r tfb.zip ./tfb/
```

Sur votre ordinateur, dans un terminal, se placer dans le dossier ou est le notebook et exécuter:

```
tensorboard --logdir ./tfb/
```

Vous devriez avoir un visuel comme ci-dessous. Vous pouvez cliquer sur un sms et vous avez à droite les sms les plus proches en terme de distance cosine comme nous l'avons fait pour word2vec. Par ailleurs chaque sms est représenté par le smiley correspondant. Vous pouvez faire varier les méthodes de projection dans le panneau de gauche.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP3%20-%20Word%20Embeddings/tfb-viz.png?raw=True" width="1000">

## 📚 References

> <div id="panckhurst-2016">Panckhurst, Rachel, et al. <a href=https://hal.archives-ouvertes.fr/hal-01485560> 88milSMS. A corpus of authentic text messages in French.</a> Banque de corpus CoMeRe. Chanier T.(éd)-Ortolang: Nancy (2016).</div>

> <div id="schmidhuber-1997">Sepp Hochreiter, Jürgen Schmidhuber. <a href=https://dl.acm.org/doi/10.1162/neco.1997.9.8.1735> Long Short-Term Memory.</a> Neural Comput. 9(8): 1735-1780 (1997).</div>

> <div id="cho-2014">Kyunghyun Cho, Bart van Merrienboer, Çaglar Gülçehre, Dzmitry Bahdanau, Fethi Bougares, Holger Schwenk, Yoshua Bengio: <a href=https://doi.org/10.3115/v1/d14-1179> Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation.</a> EMNLP 2014: 1724-1734.</div>

